In [1]:
!pip install folium geopandas branca

In [4]:
import folium
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from branca.colormap import linear
import requests

In [6]:

# ---------------- CONFIGURAÇÕES ----------------
estado = "RS"  # Rio Grande do Sul
titulo_mapa = "Layout de Mapas Temático Quantitativos de Pontos"

# ---------- DADOS SIMULADOS (PONTOS ESPALHADOS NO RS) ----------
dados = {
    'nome': ['Ponto A', 'Ponto B', 'Ponto C', 'Ponto D', 'Ponto E'],
    'classe': [1, 2, 3, 4, 5],
    'lat': [-29.5, -30.2, -31.0, -30.8, -29.8],
    'lon': [-51.2, -52.5, -53.4, -54.0, -52.0]
}
df = pd.DataFrame(dados)
geometry = [Point(xy) for xy in zip(df['lon'], df['lat'])]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

# ---------- MAPA BASE (CENTRALIZADO NO RS) ----------
mapa = folium.Map(location=[-30.0, -53.0], zoom_start=6, tiles="OpenStreetMap")

# ---------- CAMADA DE PONTOS ----------
pontos = folium.FeatureGroup(name="Pontos (Classe 1 a 5)")
cores = linear.Oranges_09.scale(1, 5)

for _, row in gdf.iterrows():
    classe = row['classe']
    cor = cores(classe)
    tamanho = 4 + classe * 2
    coords = [row.geometry.y, row.geometry.x]

    folium.CircleMarker(
        location=coords,
        radius=tamanho,
        color=cor,
        fill=True,
        fill_color=cor,
        fill_opacity=1.2,
        popup=f"{row['nome']} - Classe: {classe}"
    ).add_to(pontos)

cores.caption = "Classe (1 a 5)"
cores.add_to(mapa)
pontos.add_to(mapa)

# ---------- CAMADA ESTADO ----------
url_estado = f"https://raw.githubusercontent.com/josecacciatore89/Estados_BR_IBGE/main/{estado}.geojson"
camada_estado = folium.FeatureGroup(name=f"Estado - {estado}")

try:
    geojson_estado = requests.get(url_estado).json()
    folium.GeoJson(
        geojson_estado,
        name=estado,
        style_function=lambda x: {
            "fillOpacity": 0.0,
            "color": "black",
            "weight": 2
        },
        highlight_function=lambda x: {"weight": 3, "color": "blue"},
        tooltip=estado
    ).add_to(camada_estado)
    camada_estado.add_to(mapa)
except Exception as e:
    print(f"Erro ao carregar estado {estado}: {e}")

# ---------- CAMADA MUNICÍPIOS ----------
url_municipios = f"https://raw.githubusercontent.com/josecacciatore89/municipios_BR_IBGE2023/main/{estado}_2023.geojson"
camada_municipios = folium.FeatureGroup(name=f"Municípios {estado}")

try:
    geojson_municipios = requests.get(url_municipios).json()
    folium.GeoJson(
        geojson_municipios,
        name=f"Municipios_{estado}",
        style_function=lambda x: {
            "fillOpacity": 0.0,
            "color": "black",
            "weight": 0.5
        },
        popup=folium.GeoJsonPopup(fields=["NM_MUN"]),
        tooltip=folium.GeoJsonTooltip(fields=["NM_MUN"])
    ).add_to(camada_municipios)
    camada_municipios.add_to(mapa)
except Exception as e:
    print(f"Erro ao carregar municípios do estado {estado}: {e}")

# ---------- CONTROLE DE CAMADAS ----------
folium.LayerControl(collapsed=False).add_to(mapa)

# ---------- CABEÇALHO COM TÍTULO ----------
cor_fundo = "#FFFFFF"
cor_fonte = "#000000"
html = f"""
<div style="background-color: {cor_fundo}; color: {cor_fonte}; font-family: Arial; padding: 20px; text-align: center; font-weight: bold; font-size: 20px;">
    {titulo_mapa}
</div>
"""
mapa.get_root().html.add_child(folium.Element(html))

# ---------- LOGOTIPOS ----------

left_logo = '<img src="https://raw.githubusercontent.com/josecacciatore89/imagens/main/cropped-logo.png" style="position:absolute;top:10px;left:100px;width:150px;">'
right_logo = '<img src="https://raw.githubusercontent.com/josecacciatore89/imagens/main/cropped-logo.png" style="position:absolute;top:10px;right:100px;width:150px;">'
mapa.get_root().html.add_child(folium.Element(left_logo))
mapa.get_root().html.add_child(folium.Element(right_logo))



# ---------- SALVA O MAPA ----------
mapa.save("mapa_pontos.html")
print("Mapa salvo como mapa_pontos.html")

Mapa salvo como mapa_pontos.html
